In [ ]:
from cling import cling, bash

**Listing 14.1**

Caption: A simple MPI code that sends the message ``Hello, world.'' 

In [ ]:
%%writefile Listing_14_1.cpp
#include <mpi.h>
#include <sstream>
//-----------------------------------------------------------------
int main(int argc, char **argv) {
  MPI_Init(&argc, &argv);
  int rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  int sender = 1;
  int receiver = 0;
  int tag = 16;
  if(rank == sender) {
    std::ostringstream msg;
    msg << "Hello, world, from " << sender;
    auto m = msg.str();
    std::cout << "Sender is: " << sender << std::endl;
    MPI_Send(m.c_str(), m.size()+1, MPI_CHAR, receiver, tag, MPI_COMM_WORLD);
  } else if(rank == receiver) {
    char msgbuf[100];
    MPI_Recv(msgbuf, sizeof(msgbuf), MPI_CHAR, sender, tag, MPI_COMM_WORLD, nullptr);
    std::cout << "Receiver is: " << receiver << std::endl;
    std::cout << "Message is: " << msgbuf << std::endl;
  }
  MPI_Finalize();
}


In [ ]:
%%bash
mpicxx -I . -o Listing_14_1.exe Listing_14_1.cpp
mpirun --bind-to none -np 3 ./Listing_14_1.exe

**Listing 14.2**

Caption: A simple MPI code that sends the message ``Hello, world.'' 

In [ ]:
%%writefile Listing_14_2.cpp
#include <iostream>

#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <sstream>
//-----------------------------------------------------------------
std::string msg() {
  std::ostringstream msg;
  msg << "Hello, world, from " << hpx::find_here();
  return msg.str();
}

// Make it possible to call msg() on another location.
HPX_PLAIN_ACTION(msg, msg_action);

int main() {
  auto localities = hpx::find_remote_localities();
  std::cout << "Sender is: " << hpx::find_here() << std::endl;
  std::cout << "Receiver is: " << localities[0] << std::endl;
  // Run the message on locality 1 and get the result in a future.
  hpx::future<std::string> f = hpx::async<msg_action>(localities[0]);
  std::cout << "Message is: " << f.get() << std::endl;
}


In [ ]:
%%bash
hpxcxx -I . --exe=Listing_14_2.exe Listing_14_2.cpp
hpxrun.py -l 3 -t 1 ./Listing_14_2.exe

**Listing 14.3**

Caption: The struct data_server for a HPX component with a single component action.

In [ ]:
%%writefile data_server.hpp
struct data_server
    : hpx::components::component_base<data_server> {

  // Construct new instances
  data_server() {} 

  data_server(size_t size_) { 
    for(int i=0;i<size_;i++)
      data.push_back(0.0);
  }

  data_server(size_t size, double const value) { 
    for (size_t i=0;i < size; ++i)
      data.push_back(value);
  }

  // Access data
  std::vector<double> get_data() const { 
    return data;
  }

  // Every member function which has to be invoked remotely needs to be  wrapped into a component action. The macro below defines a new type 'get_data_action' which represents the (possibly remote) member function
  HPX_DEFINE_COMPONENT_DIRECT_ACTION( 
  data_server, get_data, get_data_action)

  private:
    std::vector<double> data; 
};

// Code generation via macros to expose the data_server as a component for remote access

// HPX_REGISTER_COMPONENT() exposes the component creation
typedef hpx::components::component<data_server> data_server_type;
HPX_REGISTER_COMPONENT(data_server_type, data_server) 

// HPX_REGISTER_ACTION() exposes the component member function for remote invocation.
typedef data_server::get_data_action get_data_action;
HPX_REGISTER_ACTION(get_data_action) 


**Listing 14.4**

Caption: The struct data_client for a HPX component with a single component action.

In [ ]:
%%writefile data_client.hpp
struct data_client
    : hpx::components::client_base<data_client, data_server> {
  typedef hpx::components::client_base<data_client, data_server>
      base_type;  

  data_client() {} 

  // Create new component on locality 'where' and initialize the
  // held data
  data_client(hpx::id_type where, std::size_t size,
              double initial_value)
      : base_type(hpx::new_<data_server>(where, size,
                                         initial_value)) {} 

  // Attach a future representing a (possibly remote) partition.
  data_client(hpx::future<hpx::id_type> &&id)
      : base_type(std::move(id)) {}  

  // Invoke the (remote) member function which gives us access to
  // the data. This is a pure helper function hiding the async.
  hpx::future<std::vector<double>> get_data() const { 
    return hpx::async(get_data_action(), get_id());
  }
};


**Listing 14.5**

Caption: Example for component and component actions.

In [ ]:
%%writefile Listing_14_5.cpp
#include <vector>
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>

#include <data_server.hpp>
#include <data_client.hpp>
//-----------------------------------------------------------------

int main(int args, char **argv) {

  std::vector<hpx::id_type> localities = hpx::find_all_localities(); 

  // Generate ten double values initialized with 1
  // on the local locality, i.e. localities[0].
  data_client local = data_client(localities[0],10,1); 
  hpx::future<std::vector<double>> data_local = local.get_data(); 

  // Generate ten double values initialized with 2
  // on the first remote locality
  if (localities.size() > 0) {
  	  data_client remote(localities[1],20,2); 
	  hpx::future<std::vector<double>> data_remote = remote.get_data(); 
	  std::vector<double> d_remote = data_remote.get(); 
      std::cout << "remote: " << d_remote[1] << std::endl;
  }

  std::vector<double> d_local = data_local.get(); 
  std::cout << "local: " << d_local[1] << std::endl;

  return EXIT_SUCCESS;
}


In [ ]:
%%bash
hpxcxx -I . --exe=Listing_14_5.exe Listing_14_5.cpp
hpxrun.py -l 3 -t 1 ./Listing_14_5.exe

**Listing 14.6**

Caption: Functions to receive topology information for distributed HPX applications.

In [ ]:
%%cling
#include <hpx/hpx.hpp>
#include <iostream>
//-----------------------------------------------------------------
// Get the number of OS threads
std::cout << hpx::get_os_thread_count() << std::endl; 

// Get the number of all localities
std::cout << hpx::get_num_localities().get() << std::endl; 

// Get all localities
std::cout << hpx::find_all_localities().size() << std::endl; 

// Get all remote localities
std::cout << hpx::find_remote_localities().size() << std::endl; 

// Get the global address of the locality the function is executed, the current locality.
hpx::id_type here = hpx::find_here(); 


**Listing 14.7**

Caption: Using HPX's serialization for remote function calls.

In [ ]:
%%writefile Listing_14_7.cpp
#include <cstdlib>
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <iostream>
//-----------------------------------------------------------------
struct data {
private:
  typedef hpx::serialization::serialize_buffer<double>
      buffer_type; 

public:
  data() : label("empty") {}  
  data(const std::string& label_,size_t size) 
      : label(label_),
        data_(std::allocator<double>().allocate(size), size, buffer_type::take) {}
  data(const std::string& label_,size_t size, double* data__) 
      : label(label_),
        data_(std::allocator<double>().allocate(size), size, buffer_type::take) {
    for (size_t i = 0; i < size; i++) data_[i] = data__[i];
  }
  data(std::string label_,size_t size, double data) 
      : label(label_), data_(std::allocator<double>().allocate(size), size, buffer_type::take) {
    for (size_t i = 0; i < size; i++) data_[i] = data;
  }
  data(std::string label_,size_t size, std::function<double(int)> f) 
      : label(label_), data_(std::allocator<double>().allocate(size), size, buffer_type::take) {
    for (size_t i = 0; i < size; i++) data_[i] = f(i);
  }
  double& operator[](size_t id) { return data_[id]; } 
  double operator[](size_t id) const { return data_[id]; } 

  size_t size() const { return data_.size(); } 

  std::string get_label() const { return label; } 
  void set_label(const std::string s) { label = s; } 

 private:
  std::string label;
  buffer_type data_; 

  friend class hpx::serialization::access; 

  template <typename Archive>  
  void serialize(Archive& ar, const unsigned int) {
    ar & label;
    ar & data_;
  }
};

static data square(data d) { 
  for (size_t i =0; i <  d.size() ; i++) d[i] = d[i] * d[i];
  d.set_label(d.get_label() + " squared");
  return d;
}


std::ostream& operator<<(std::ostream& o, const data& d) { 
    o << "data('" << d.get_label() << "',{";
    for(size_t i=0;i<d.size();i++) {
        if(i > 0) o << ", ";
        o << d[i];
    }
    return o << "})";
}

HPX_PLAIN_ACTION(square,square_action);  

int main(int args, char** argv) {
    data d = data("stats",10,[](int n) { return n; });  

    auto remote = hpx::find_remote_localities (); 
    if ( remote.size () > 0) {
      hpx::future<data> future = hpx::async<square_action>(remote[0],d); 
      data result = future.get();
      std::cout << result << std::endl;
    }
  return EXIT_SUCCESS;
}


In [ ]:
%%bash
hpxcxx -I . --exe=Listing_14_7.exe Listing_14_7.cpp
hpxrun.py -l 3 -t 1 ./Listing_14_7.exe